In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
  #  for filename in filenames:
       # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf

In [4]:
vgg=tf.keras.applications.VGG16(include_top=False,input_shape=(224,224,3))

58892288/58889256 [==============================] - 0s 0us/step


In [5]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
for layer in vgg.layers:
    layer.trainable=False

In [ ]:
#making trainale parameter zero,
#so that we can add our trainable parameters.

In [7]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten,Dense

In [ ]:
# to add flatten layer we use 'output' &  on this flatten layer we are adding dense layer

In [12]:
flatten_layer=Flatten()(vgg.output)
final_layer=Dense(2,activation='softmax')(flatten_layer)

In [16]:
flatten_layer.shape

TensorShape([None, 25088])

In [17]:
final_layer.shape

TensorShape([None, 2])

In [ ]:
#To assemble vgg model and our model

In [18]:
model=Model(inputs=vgg.input,outputs=final_layer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [20]:
filenames=os.listdir('/kaggle/input/dogs-vs-cats/train/train')
classes=[]
for file in filenames:
    classes.append(file.split('.')[0])

In [22]:
df=pd.DataFrame({'Filename':filenames,'class':classes})

In [23]:
df.head()

,Filename,class
0,cat.12461.jpg,cat
1,dog.3443.jpg,dog
2,dog.7971.jpg,dog
3,dog.10728.jpg,dog
4,dog.1942.jpg,dog


In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [25]:
data_gen=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,horizontal_flip=True,rescale=1/255)

In [26]:
train_data=data_gen.flow_from_dataframe(df,'/kaggle/input/dogs-vs-cats/train/train',x_col='Filename',y_col='class',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 25000 validated image filenames belonging to 2 classes.


In [28]:
train_data.class_indices

{'cat': 0, 'dog': 1}

In [30]:
model.fit_generator(train_data,steps_per_epoch=len(train_data),epochs=4)

Epoch 1/4
782/782 [==============================] - 456s 583ms/step - loss: 0.2384 - accuracy: 0.9096
Epoch 2/4
782/782 [==============================] - 398s 508ms/step - loss: 0.2145 - accuracy: 0.9217
Epoch 3/4
782/782 [==============================] - 400s 511ms/step - loss: 0.2162 - accuracy: 0.9244
Epoch 4/4
782/782 [==============================] - 401s 513ms/step - loss: 0.2014 - accuracy: 0.9305


In [31]:
model.save('VGG_model.h5')

In [32]:
new_model=tf.keras.models.load_model('./VGG_model.h5')

In [33]:
import cv2

In [36]:
def get_cat_or_dog(filepath):
    img0=cv2.imread(filepath)
    img0=cv2.resize(img0,(224,224))/255
    class_code=new_model.predict(img0.reshape(1,224,224,3)).argmax()
    return 'CAT' if class_code==0 else 'DOG'

In [41]:
get_cat_or_dog('../input/dogs-vs-cats/test/test/1005.jpg')

'CAT'